# Active Learning Notebook

In [17]:
import random
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle

from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.ensemble import RandomForestClassifier as rf

from modAL.models import ActiveLearner
from modAL.uncertainty import uncertainty_sampling

## Init config

In [18]:
filename = 'df_150000_20.pkl'
multiclass = True

## Data Pre-processing

In [19]:
df = pd.read_pickle(filename)
X = df.iloc[:,:df.shape[1]-2]
y = df.iloc[:,df.shape[1]-1]

In [20]:
if multiclass == False:
    y = y.replace([2,3],1)

In [23]:
X_train, X_test, y_train, y_test = train_test_split(X.to_numpy(),
                                                    y.to_numpy(),
                                                    test_size=0.30,
                                                    stratify=y,
                                                    random_state=77)

In [24]:
print(f'X train shape:{X_train.shape}')
print(f'y train shape:{y_train.shape}')
print(f'X test shape:{X_test.shape}')
print(f'y test shape:{y_test.shape}')

X train shape:(105000, 18)
y train shape:(105000,)
X test shape:(45000, 18)
y test shape:(45000,)


In [27]:
# Isolate our examples for our labeled dataset.
n_labeled_examples = X_train.shape[0]
training_indices = np.random.randint(low=0,
                                     high=n_labeled_examples + 1,
                                     size=100)

In [28]:
# Initial Seed
X_init = X_train[training_indices]
y_init = y_train[training_indices]

In [29]:
# Pool
# Delete same
X_pool = np.delete(X_train, training_indices, axis=0)
y_pool = np.delete(y_train, training_indices, axis=0)

# Make Pool
n_labeled_examples = X_pool.shape[0]
pool_indices = np.random.randint(low=0, high=n_labeled_examples + 1, size=10000)
X_pool = X_pool[pool_indices]
y_pool = y_pool[pool_indices]

In [30]:
print(f'X init shape:{X_init.shape}')
print(f'y init shape:{y_init.shape}')
print(f'X pool shape:{X_pool.shape}')
print(f'y pool shape:{y_pool.shape}')

X init shape:(100, 18)
y init shape:(100,)
X pool shape:(10000, 18)
y pool shape:(10000,)


In [31]:
rf = rf()
learner = ActiveLearner(estimator=rf, X_training=X_init, y_training=y_init)

In [32]:
predictions = learner.predict(X_test)

In [34]:
init_score = learner.score(X_test, y_test)
print(f'Initial Score: {init_score}')

Initial Score: 0.9623555555555555


In [5]:
init_size = 100
n_labeled_examples = X_train.shape[0]
training_indices = np.random.randint(low=0, high=n_labeled_examples + 1, size=init_size)

# Initial 
X_init = X_train[training_indices]
y_init = y_train[training_indices]

# Pool (Unlabled)
X_pool = np.delete(X_train, training_indices, axis=0)
y_pool = np.delete(y_train, training_indices, axis=0)

print(f'X init shape:{X_init.shape}')
print(f'y init shape:{y_init.shape}')
print(f'X pool shape:{X_pool.shape}')
print(f'y pool shape:{y_pool.shape}')

clf = rf()
learner = ActiveLearner(estimator=clf, X_training=X_init, y_training=y_init)

predictions = learner.predict(X_test)

init_score = learner.score(X_test, y_test)
print(f'Initial Score:{init_score}')

performance_history = [init_score]

# Allow our model to query our unlabeled dataset for the most
# informative points according to our query strategy (uncertainty sampling).
for index in range(3000):
    query_index, query_instance = learner.query(X_pool)

    # Teach our ActiveLearner model the record it has requested.
    X, y = X_pool[query_index].reshape(1, -1), y_pool[query_index].reshape(1, )
    learner.teach(X=X, y=y)

    # Remove the queried instance from the unlabeled pool.
    X_pool, y_pool = np.delete(X_pool, query_index, axis=0), np.delete(y_pool, query_index)

    # Calculate and report our model's accuracy.
    model_accuracy = learner.score(X_test, y_test)
    print('Accuracy after query {n}: {acc:0.4f}'.format(n=index + 1, acc=model_accuracy))

    # Save our model's performance for plotting.
    performance_history.append(model_accuracy)

file_name = str(init_size)+'_uncert.pkl'
file = open(file_name, 'wb')
pickle.dump(performance_history, file)
file.close()

X init shape:(200, 24)
y init shape:(200,)
X pool shape:(130727, 24)
y pool shape:(130727,)
Initial Score:0.9562482178500142
Accuracy after query 1: 0.9561
Accuracy after query 2: 0.9536
Accuracy after query 3: 0.9544
Accuracy after query 4: 0.9556
Accuracy after query 5: 0.9550
Accuracy after query 6: 0.9551
Accuracy after query 7: 0.9554
Accuracy after query 8: 0.9568
Accuracy after query 9: 0.9586
Accuracy after query 10: 0.9577
Accuracy after query 11: 0.9585
Accuracy after query 12: 0.9564
Accuracy after query 13: 0.9556
Accuracy after query 14: 0.9580
Accuracy after query 15: 0.9587
Accuracy after query 16: 0.9589
Accuracy after query 17: 0.9594
Accuracy after query 18: 0.9596
Accuracy after query 19: 0.9597
Accuracy after query 20: 0.9596
Accuracy after query 21: 0.9597
Accuracy after query 22: 0.9600
Accuracy after query 23: 0.9598
Accuracy after query 24: 0.9597
Accuracy after query 25: 0.9598
Accuracy after query 26: 0.9602
Accuracy after query 27: 0.9600
Accuracy after query

Accuracy after query 249: 0.9806
Accuracy after query 250: 0.9809
Accuracy after query 251: 0.9816
Accuracy after query 252: 0.9809
Accuracy after query 253: 0.9809
Accuracy after query 254: 0.9808
Accuracy after query 255: 0.9813
Accuracy after query 256: 0.9815
Accuracy after query 257: 0.9816
Accuracy after query 258: 0.9817
Accuracy after query 259: 0.9815
Accuracy after query 260: 0.9814
Accuracy after query 261: 0.9818
Accuracy after query 262: 0.9819
Accuracy after query 263: 0.9814
Accuracy after query 264: 0.9815
Accuracy after query 265: 0.9815
Accuracy after query 266: 0.9812
Accuracy after query 267: 0.9815
Accuracy after query 268: 0.9816
Accuracy after query 269: 0.9816
Accuracy after query 270: 0.9816
Accuracy after query 271: 0.9816
Accuracy after query 272: 0.9816
Accuracy after query 273: 0.9820
Accuracy after query 274: 0.9819
Accuracy after query 275: 0.9821
Accuracy after query 276: 0.9819
Accuracy after query 277: 0.9818
Accuracy after query 278: 0.9817
Accuracy a

Accuracy after query 498: 0.9924
Accuracy after query 499: 0.9920
Accuracy after query 500: 0.9925
Accuracy after query 501: 0.9926
Accuracy after query 502: 0.9924
Accuracy after query 503: 0.9922
Accuracy after query 504: 0.9924
Accuracy after query 505: 0.9923
Accuracy after query 506: 0.9924
Accuracy after query 507: 0.9922
Accuracy after query 508: 0.9925
Accuracy after query 509: 0.9924
Accuracy after query 510: 0.9926
Accuracy after query 511: 0.9927
Accuracy after query 512: 0.9923
Accuracy after query 513: 0.9925
Accuracy after query 514: 0.9926
Accuracy after query 515: 0.9926
Accuracy after query 516: 0.9926
Accuracy after query 517: 0.9927
Accuracy after query 518: 0.9922
Accuracy after query 519: 0.9926
Accuracy after query 520: 0.9927
Accuracy after query 521: 0.9923
Accuracy after query 522: 0.9925
Accuracy after query 523: 0.9925
Accuracy after query 524: 0.9927
Accuracy after query 525: 0.9924
Accuracy after query 526: 0.9925
Accuracy after query 527: 0.9928
Accuracy a

Accuracy after query 747: 0.9936
Accuracy after query 748: 0.9939
Accuracy after query 749: 0.9937
Accuracy after query 750: 0.9937
Accuracy after query 751: 0.9936
Accuracy after query 752: 0.9934
Accuracy after query 753: 0.9936
Accuracy after query 754: 0.9940
Accuracy after query 755: 0.9938
Accuracy after query 756: 0.9937
Accuracy after query 757: 0.9936
Accuracy after query 758: 0.9936
Accuracy after query 759: 0.9934
Accuracy after query 760: 0.9935
Accuracy after query 761: 0.9938
Accuracy after query 762: 0.9936
Accuracy after query 763: 0.9938
Accuracy after query 764: 0.9940
Accuracy after query 765: 0.9939
Accuracy after query 766: 0.9938
Accuracy after query 767: 0.9938
Accuracy after query 768: 0.9939
Accuracy after query 769: 0.9938
Accuracy after query 770: 0.9939
Accuracy after query 771: 0.9938
Accuracy after query 772: 0.9938
Accuracy after query 773: 0.9938
Accuracy after query 774: 0.9939
Accuracy after query 775: 0.9939
Accuracy after query 776: 0.9937
Accuracy a

Accuracy after query 996: 0.9945
Accuracy after query 997: 0.9944
Accuracy after query 998: 0.9945
Accuracy after query 999: 0.9944
Accuracy after query 1000: 0.9948
Accuracy after query 1001: 0.9945
Accuracy after query 1002: 0.9944
Accuracy after query 1003: 0.9942
Accuracy after query 1004: 0.9946
Accuracy after query 1005: 0.9945
Accuracy after query 1006: 0.9945
Accuracy after query 1007: 0.9947
Accuracy after query 1008: 0.9945
Accuracy after query 1009: 0.9949
Accuracy after query 1010: 0.9944
Accuracy after query 1011: 0.9945
Accuracy after query 1012: 0.9945
Accuracy after query 1013: 0.9946
Accuracy after query 1014: 0.9943
Accuracy after query 1015: 0.9948
Accuracy after query 1016: 0.9948
Accuracy after query 1017: 0.9947
Accuracy after query 1018: 0.9946
Accuracy after query 1019: 0.9946
Accuracy after query 1020: 0.9944
Accuracy after query 1021: 0.9947
Accuracy after query 1022: 0.9946
Accuracy after query 1023: 0.9947
Accuracy after query 1024: 0.9944
Accuracy after que

...

Accuracy after query 1808: 0.9949
Accuracy after query 1809: 0.9952
Accuracy after query 1810: 0.9950
Accuracy after query 1811: 0.9951
Accuracy after query 1812: 0.9951
Accuracy after query 1813: 0.9950
Accuracy after query 1814: 0.9950
Accuracy after query 1815: 0.9951
Accuracy after query 1816: 0.9951
Accuracy after query 1817: 0.9951
Accuracy after query 1818: 0.9950
Accuracy after query 1819: 0.9951
Accuracy after query 1820: 0.9952
Accuracy after query 1821: 0.9950
Accuracy after query 1822: 0.9950
Accuracy after query 1823: 0.9951
Accuracy after query 1824: 0.9951
Accuracy after query 1825: 0.9950
Accuracy after query 1826: 0.9950
Accuracy after query 1827: 0.9950
Accuracy after query 1828: 0.9951
Accuracy after query 1829: 0.9952
Accuracy after query 1830: 0.9952
Accuracy after query 1831: 0.9951
Accuracy after query 1832: 0.9951
Accuracy after query 1833: 0.9951
Accuracy after query 1834: 0.9952
Accuracy after query 1835: 0.9952
Accuracy after query 1836: 0.9951
Accuracy after

Accuracy after query 2049: 0.9951
Accuracy after query 2050: 0.9952
Accuracy after query 2051: 0.9952
Accuracy after query 2052: 0.9952
Accuracy after query 2053: 0.9951
Accuracy after query 2054: 0.9951
Accuracy after query 2055: 0.9951
Accuracy after query 2056: 0.9951
Accuracy after query 2057: 0.9952
Accuracy after query 2058: 0.9951
Accuracy after query 2059: 0.9952
Accuracy after query 2060: 0.9952
Accuracy after query 2061: 0.9951
Accuracy after query 2062: 0.9952
Accuracy after query 2063: 0.9951
Accuracy after query 2064: 0.9952
Accuracy after query 2065: 0.9950
Accuracy after query 2066: 0.9953
Accuracy after query 2067: 0.9952
Accuracy after query 2068: 0.9951
Accuracy after query 2069: 0.9952
Accuracy after query 2070: 0.9953
Accuracy after query 2071: 0.9952
Accuracy after query 2072: 0.9952
Accuracy after query 2073: 0.9951
Accuracy after query 2074: 0.9952
Accuracy after query 2075: 0.9952
Accuracy after query 2076: 0.9951
Accuracy after query 2077: 0.9952
Accuracy after

Accuracy after query 2290: 0.9950
Accuracy after query 2291: 0.9951
Accuracy after query 2292: 0.9952
Accuracy after query 2293: 0.9953
Accuracy after query 2294: 0.9954
Accuracy after query 2295: 0.9952
Accuracy after query 2296: 0.9952
Accuracy after query 2297: 0.9953
Accuracy after query 2298: 0.9952
Accuracy after query 2299: 0.9952
Accuracy after query 2300: 0.9952
Accuracy after query 2301: 0.9953
Accuracy after query 2302: 0.9950
Accuracy after query 2303: 0.9953
Accuracy after query 2304: 0.9953
Accuracy after query 2305: 0.9953
Accuracy after query 2306: 0.9953
Accuracy after query 2307: 0.9951
Accuracy after query 2308: 0.9953
Accuracy after query 2309: 0.9953
Accuracy after query 2310: 0.9952
Accuracy after query 2311: 0.9952
Accuracy after query 2312: 0.9950
Accuracy after query 2313: 0.9953
Accuracy after query 2314: 0.9953
Accuracy after query 2315: 0.9954
Accuracy after query 2316: 0.9950
Accuracy after query 2317: 0.9953
Accuracy after query 2318: 0.9952
Accuracy after

Accuracy after query 2531: 0.9951
Accuracy after query 2532: 0.9952
Accuracy after query 2533: 0.9952
Accuracy after query 2534: 0.9951
Accuracy after query 2535: 0.9951
Accuracy after query 2536: 0.9951
Accuracy after query 2537: 0.9952
Accuracy after query 2538: 0.9951
Accuracy after query 2539: 0.9950
Accuracy after query 2540: 0.9953
Accuracy after query 2541: 0.9951
Accuracy after query 2542: 0.9952
Accuracy after query 2543: 0.9951
Accuracy after query 2544: 0.9951
Accuracy after query 2545: 0.9951
Accuracy after query 2546: 0.9951
Accuracy after query 2547: 0.9951
Accuracy after query 2548: 0.9952
Accuracy after query 2549: 0.9950
Accuracy after query 2550: 0.9950
Accuracy after query 2551: 0.9950
Accuracy after query 2552: 0.9952
Accuracy after query 2553: 0.9951
Accuracy after query 2554: 0.9950
Accuracy after query 2555: 0.9951
Accuracy after query 2556: 0.9952
Accuracy after query 2557: 0.9951
Accuracy after query 2558: 0.9951
Accuracy after query 2559: 0.9952
Accuracy after

Accuracy after query 2772: 0.9952
Accuracy after query 2773: 0.9951
Accuracy after query 2774: 0.9951
Accuracy after query 2775: 0.9951
Accuracy after query 2776: 0.9950
Accuracy after query 2777: 0.9951
Accuracy after query 2778: 0.9952
Accuracy after query 2779: 0.9952
Accuracy after query 2780: 0.9952
Accuracy after query 2781: 0.9950
Accuracy after query 2782: 0.9951
Accuracy after query 2783: 0.9952
Accuracy after query 2784: 0.9952
Accuracy after query 2785: 0.9951
Accuracy after query 2786: 0.9951
Accuracy after query 2787: 0.9951
Accuracy after query 2788: 0.9951
Accuracy after query 2789: 0.9952
Accuracy after query 2790: 0.9950
Accuracy after query 2791: 0.9950
Accuracy after query 2792: 0.9952
Accuracy after query 2793: 0.9951
Accuracy after query 2794: 0.9952
Accuracy after query 2795: 0.9952
Accuracy after query 2796: 0.9951
Accuracy after query 2797: 0.9951
Accuracy after query 2798: 0.9951
Accuracy after query 2799: 0.9950
Accuracy after query 2800: 0.9952
Accuracy after